<a href="https://colab.research.google.com/github/mnida/spectogram-gpt-features/blob/main/Spectogram_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
import numpy as np
from functools import partial
from typing import List, Optional, Union
import einops
from fancy_einsum import einsum
import sklearn
import matplotlib.pyplot as plt
import sys
import os

sys.path.append("..")

from sae_training.utils import LMSparseAutoencoderSessionloader
from sae_analysis.visualizer import data_fns, html_fns
from sae_analysis.visualizer.data_fns import get_feature_data, FeatureData

if torch.backends.mps.is_available():
    device = "mps" 
else:
    device = "cuda" if torch.cuda.is_available() else "cpu"

torch.set_grad_enabled(False)

def imshow(x, **kwargs):
    x_numpy = utils.to_numpy(x)
    px.imshow(x_numpy, **kwargs).show()

In [10]:
print(torch.device(device))


mps


Our goal here is to create a mel spectrogram type of visualization of GPT-2 Sparse Autoencoder (SAE) different features. The goal is to visualization how features are alike or different.

Inspired entirely by @thesephist's tweet: https://x.com/thesephist/status/1754322902210793727?s=20

Using the SAE from Joseph Isaac Bloom:
https://www.alignmentforum.org/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream


Load in the Sparse Autoencoder.

In [33]:

path = "artifacts/sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_6144:v2/1200001024_sparse_autoencoder_gpt2-small_blocks.10.hook_resid_pre_6144.pt"
model, sparse_autoencoder, activations_loader = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  mps


/Users/mnida/miniconda3/envs/mats_sae_training/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Dataset is not tokenized! Updating config.


In [18]:
with open('data.txt', 'r') as file:
    text_data = file.read()
tokenized_text = model.tokenizer.encode(text_data)
tokenized_text_tensor = torch.tensor(tokenized_text).unsqueeze(0)

Token indices sequence length is longer than the specified maximum sequence length for this model (1900 > 1024). Running this sequence through the model will result in indexing errors


In [23]:
# Truncate or pad the tokenized text to fit the model's input size
max_input_size = 1024  # Replace this with your model's max input size
if len(tokenized_text_tensor[0]) > max_input_size:
    tokenized_text_tensor = tokenized_text_tensor[:, :max_input_size]
elif len(tokenized_text_tensor[0]) < max_input_size:
    padding = torch.zeros((1, max_input_size - len(tokenized_text_tensor[0])))
    tokenized_text_tensor = torch.cat((tokenized_text_tensor, padding), dim=1)

In [26]:
print(tokenized_text_tensor.shape)
print(model)

torch.Size([1, 1024])
HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out)

In [35]:

with torch.no_grad():
    batch_tokens = activations_loader.get_batch_tokens()
    _, cache = model.run_with_cache(batch_tokens, prepend_bos=True)
    print(cache[sparse_autoencoder.cfg.hook_point] is None)
    sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(
        cache[sparse_autoencoder.cfg.hook_point]
    )
    del cache

False


AttributeError: 'NoneType' object has no attribute 'sum'

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=1, perplexity=30, n_iter=300)
tsne_results = tsne.fit_transform(feature_acts.detach().numpy())

ModuleNotFoundError: No module named 'sklearn'

## Visualize the Spectogram with Sklearn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
times = np.arange(len(tsne_results))  # Assuming each feature vector corresponds to a time token
frequencies = tsne_results.flatten()  # Your 1D t-SNE mapped to frequencies

plt.figure(figsize=(10, 4))
plt.scatter(times, frequencies, c=frequencies, cmap='viridis')  # Color by frequency for visualization
plt.colorbar(label='Frequency')
plt.xlabel('Time (tokens)')
plt.ylabel('Frequency (Hz)')
plt.title('Spectrogram Representation')
plt.show()